# Exercise 3 - Introduction to Gemini API and prompting
Google makes their models available via the [Gemini API](https://cloud.google.com/vertex-ai/generative-ai/docs/start/quickstarts/quickstart-multimodal#python) and multiple SDKs.
In this exercise, we will explore how to use the Python SDK to interact with these models.


If you haven't done so already, it's time to authenticate Google cloud.
<details>
<summary>Steps for authenticating Google Cloud</summary>

1. Go to the [Google Cloud Console](https://console.cloud.google.com/).
2. In the search bar at the top, search for the project ID used for this training. Your trainer will confirm what this is, but it probably starts with `academy-llm-applications-*`!
3. In your Codespace terminal, run `gcloud auth login` and log in to your Google account.
4. In the same terminal, run `gcloud auth application-default login` and log in to your Google account.
5. Set the project ID by running `gcloud config set project <PROJECT_ID>` in the terminal, replacing `<PROJECT_ID>` with the project ID from step 2.
6. Set the quota project by running `gcloud auth application-default set-quota-project <PROJECT_ID>` in the terminal, again replacing `<PROJECT_ID>` with the project ID from step 2.

</details>


Let's start by initiating the chat model. We will use the `vertexai` package for this to connect to Google Cloud and the Vertexi AI API's.

In [2]:
import os
import vertexai
from vertexai.generative_models import GenerativeModel, Part, Content
from IPython.display import Image, display
# load environment variables
from dotenv import load_dotenv
load_dotenv()

vertexai.init(
    project=os.environ["GCP_PROJECT_ID"],
    # location=os.environ["GCP_LOCATION"]
)

We are now able to instantiate a Generative model. Let's instantiate Gemini 1.5 flash ⚡️.

In [4]:

model = GenerativeModel("gemini-1.5-flash-002")

print(f"LLM region: {model._location}")

LLM region: us-central1


This model we can use to generate some content. Let's get some ideas for your tonight's dinner.

In [ ]:

response = model.generate_content(
    "Give me some ideas for dinner tonight."
)

print(response.text)

That's great! We now verified we could successfully connect to Gemini's API and generate text 🎉.

## Exercise 3a: Vertex AI Gemini API - capabilities walkthrough
Let's get familiar with features of the Gemini API. We will cover the following by showing you interactive code examples and explanations of each feature:

1. Multimodal generation
2. Using system prompts
3. Using content generation parameters (temperature, etc)

### 1. Multimodal generation
Gemini models are multimodal, meaning they can process different types of data, such as text, images, audio, and video. Let's explore how to use this capability with an image. We will be using a test image from Google Cloud Storage bucket, but you can replace it with a local path to an image as well. 

First, let's select an image to use and take a look at it!

In [ ]:
IMAGE_URL = "https://storage.googleapis.com/cloud-samples-data/generative-ai/image/scones.jpg"

# Display the image
display(Image(url=IMAGE_URL, width=500))


We'll need to prepare the image for use with model.

In [ ]:
image = Part.from_uri(IMAGE_URL, mime_type="image/jpeg")

Now we can use our previously initialized model, together with a prompt, to get a description of the image we just prepared.

In [ ]:
prompt = "What is this and can you tell me some interesting fact about it?"

response = model.generate_content(
    [prompt, image]
)

print(response.text)

As you can see, the model successfully understood the context of the image and gave us useful information. This showcases how to use the Gemini API for multimodal inputs. 

Now let's move on to system prompts!

### 2. Using system prompts
System prompts allow you to set the context and behaviour of the model. They are useful for creating specific personas or setting specific guidelines for the model to follow. Add below a system instruction when instantiating the model.

Take a look at the [documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/system-instructions) for some tips on writing system instructions.

In [ ]:
system_prompt = "You are a helpful and concise assistant that summarizes the input in three bullet points."
prompt = "The French Revolution was a period of social and political upheaval in late 1700's France. It eventually toppled the monarchy and led to the rise of Napoleon Bonaparte."

# YOUR CODE HERE START
model = GenerativeModel("gemini-1.5-flash-002", system_instruction = [system_prompt])
# YOUR CODE HERE END

response = model.generate_content(
    prompt,    
)

print(response.text)

As you can see, the model followed the system prompt and returned the summary in three bullet points. System prompts are a powerful way to control the model's behavior and to tailor it to your specific needs. 


### 3. Using content generation parameters
The Gemini API allows you to control the generation of text using several parameters such as `temperature`, `top_p` and `top_k`. These parameters affect the randomness and diversity of the generated content. Let's see what each of the do:

*   `temperature`: Controls the randomness of the output. Higher values result in more random outputs and lower values will result in more deterministic outputs. The suggested range is 0.0 to 1.0 but higher values can also be used (up to 2.0).
*   `top_p`: Controls the cumulative probability of the tokens selected. Lower values tend to lead to more focused generation, while higher values can result in more diverse outputs.
*   `top_k`: Controls the number of tokens that can be selected. Higher values lead to more tokens being considered.

Let's explore how temperature affects the model's output.

In [ ]:
prompt = "Tell me something about the sky."
print("Temperature 0.2")
response = model.generate_content(
    prompt,
    generation_config = {
        # YOUR CODE HERE START
        "temperature": 0.2
        # YOUR CODE HERE END
    }
)
print(response.text)

print("\nTemperature 1.0")
response = model.generate_content(
    prompt,
    # YOUR CODE HERE START
    generation_config = {"temperature": 1.0}
    # YOUR CODE HERE END
)
print(response.text)

Try also re-running the cell several times to see how random the output of the model is. As you can see, the output of the model changes based on the temperature. A lower temperature makes the output more deterministic and focused, while a higher temperature makes the output more creative and diverse.

You can experiment with the `top_p` and `top_k` parameters as well. 

This concludes this introduction into the Google Gemini Vertex AI API. You've learned how to connect to the API, generate text and process multimodal content, control the behaviour of the models with system prompts and control the output generation using temperature.
 

## Optional: Exercise 3b - base model vs instruction fine-tuned model
Let's make a CLI-based version of Gemini.

Below, you will find some boilerplate code that takes care of asking the user for input and printing the conversation.
However, there are still a few things missing like:
- The content of the inital system prompt.
- The API call to the model, which leads to a response from the chatbot based on the conversation so far.


Can you finish it?

<mark>Note: The text box should appear at the top of your IDE (VS Code) window.</mark>

In [ ]:
import time

# The initial system prompt.
messages = [
    Content(role='user', parts=[
        Part.from_text('Hi there'),
    ]),

    # pass context to the model, simulating a system prompt, by mimicking a model response.
    Content(role='model', parts=[
        Part.from_text(
            'Hi! I am an AI assistant that helps people with their every day tasks. How can I help you today?')
    ])
]

chat = model.start_chat(history=messages)

while True:
    # Get the user's input.
    user_message = input('User:').strip()

    # Check whether to continue.
    if len(user_message) == 0 or user_message == "exit":
        print("exiting...")
        break
    print(1)

    # Send the user's message to the model.
    # YOUR CODE HERE START
    response = chat.send_message(user_message,
                                 generation_config={"temperature": 1.0})
    # YOUR CODE HERE END

    assistant_message = response.text
    print(2)

    # Print the user input and response.
    print("User:", user_message)
    print("AI:", assistant_message)

    # We need to wait a little bit of time for the text to render.
    time.sleep(0.5)

---